In [9]:
import pymongo
import os
import json

In [6]:
MONGO_URI = os.environ.get("MONGO_URI", "mongodb://localhost:27017")
MONGO_DB_NAME = os.environ.get("MONGO_DB_NAME", "meuParlamento")
WORKING_DIR = os.environ.get("WORKING_DIR", "./")


In [ ]:
mongo_conn=pymongo.MongoClient(MONGO_URI)
mongo_conn_database = mongo_conn[MONGO_DB_NAME]

In [8]:
# na task de coleta da proposta, é preciso salvar tambem as propostas que ainda nao foram votadas.
# - salvar em uma coleção separada
# - ou salvar na mesma coleção, mas com um flag apontando pra isso

In [45]:
from meuparlamento.harvester import PortugueseParlamentProposalsHarvester
from meuparlamento.readability import ReadabilityScore
from meuparlamento.pdf import PDFReader 
from meuparlamento.nlp import ContentSummarizer
from meuparlamento.utils import translate_comission_category

# -*- coding: UTF-8 -*-

from lxml import html
import requests
from datetime import datetime
from meuparlamento.utils import  parse_original_document_url
from urllib.parse import urlparse
import re
import logging
logger = logging.getLogger(__name__)

class PortugueseParlamentProposalsHarvester(object):
    
    def __init__(self, config):
        self.config = config

    def _handle_fetch_proposal_response(self, response):
        print("_handle_fetch_proposal_response")
        print("response.status_code",response.status_code)
        print("response.url",response.url)
        # if(response.status_code == 200):

        # print("response.content",response.content)

        proposal = self._parse_html(response.content)


        if(proposal):
            # set origin (arquivo.pt | parlamento.pt)
            proposal["source_domain"] = urlparse(response.url).netloc
            
            proposal["url"] = response.url
            proposal["BID"] = response.request.path_url.split("=")[1]

            # TODO refactor this
            if("arquivo.pt" in proposal["pdfLink"]):
                proposal["pdfLink"] = parse_original_document_url(proposal["pdfLink"])

            return proposal
        else:
            return None
                
    def fetch_proposal(self, bid):
        proposal_url = self.config.proposal_url_template + str(bid)

        logger.info(proposal_url)
        
        response = requests.get(proposal_url)
        if(response.status_code == 200):
            return self._handle_fetch_proposal_response(response)

        else:
            logger.info("Failed request. Try web archive")
            logger.info(proposal_url)
            proposal_url =  "{webarchive_url_replay}/{url}".format(webarchive_url_replay=self.config.webarchive_url_replay,
                                                                   url=proposal_url)
            response = requests.get(proposal_url)

            return self._handle_fetch_proposal_response(response)

    def _clean_partial_votes(self, votes):
        
        for idx, vote in enumerate(votes):
            votes[idx] = re.sub(r"\d+\W+", '', vote).strip()

        return votes

    def _parse_vote_type(self):
        pass

    def _parse_voting_results(self, votes_element):
        votacoes = votes_element

        votos_contra = []
        votos_afavor = []
        votos_abstencao = []
        votos_ausencia = []
        
        if(len(votacoes) > 0):
            votacoes = votacoes[-1]
            votos = votacoes.text_content()

            CONTRA = "#contra:"
            AFAVOR = "#a_favor:"
            ABSTENCAO = "#abstencao:"
            AUSENCIA = "#ausencia:"

            votos = votos.replace("Contra:",CONTRA)
            votos = votos.replace("A Favor:",AFAVOR)
            votos = votos.replace("Abstenção:",ABSTENCAO)
            votos = votos.replace("Ausência:",AUSENCIA)
            
            if("#" not in votos):
                return

            if(CONTRA in votos):
                start_str_idx = votos.index(CONTRA) + len(CONTRA)
                end_str_idx = len(votos)
                try:
                    end_str_idx = votos.index("#",start_str_idx)
                except:
                    pass                    
                in_conta = votos[start_str_idx:end_str_idx]
                votos_contra = in_conta.strip().split(",")

            if(AFAVOR in votos):
                start_str_idx = votos.index(AFAVOR) + len(AFAVOR)
                end_str_idx = len(votos)
                try:
                    end_str_idx = votos.index("#",start_str_idx)
                except:
                    pass
                in_conta = votos[start_str_idx:end_str_idx]
                votos_afavor = in_conta.strip().split(",")

            if(ABSTENCAO in votos):
                start_str_idx = votos.index(ABSTENCAO) + len(ABSTENCAO)
                end_str_idx = len(votos)
                try:
                    end_str_idx = votos.index("#",start_str_idx)
                except:
                    pass
                in_conta = votos[start_str_idx:end_str_idx]
                votos_abstencao = in_conta.strip().split(",")  

            if(AUSENCIA in votos):
                start_str_idx = votos.index(AUSENCIA) + len(AUSENCIA)
                end_str_idx = len(votos)
                try:
                    end_str_idx = votos.index("#",start_str_idx)
                except:
                    pass
                in_conta = votos[start_str_idx:end_str_idx]
                votos_ausencia = in_conta.strip().split(",")  
              

        votos_contra = [x.strip() for x in votos_contra]
        votos_afavor = [x.strip() for x in votos_afavor]
        votos_abstencao = [x.strip() for x in votos_abstencao]
        votos_ausencia = [x.strip() for x in votos_ausencia]
        
        all_votes = {
                    "contra":self._clean_partial_votes(votos_contra),
                    "afavor":self._clean_partial_votes(votos_afavor), 
                    "abstencao":self._clean_partial_votes(votos_abstencao),
                    "ausencia":self._clean_partial_votes(votos_ausencia)
                    }
        
        return all_votes
    
    def _parse_party_from_author_name(self, s):
        return s[s.find("(")+1:s.find(")")]
    
    def _was_already_voted(self, doc):
        if(len(doc["votos"]["contra"]) > 0):
            return True
        elif(len(doc["votos"]["afavor"]) > 0):
            return True
        elif(len(doc["votos"]["abstencao"]) > 0):
            return True
        elif(len(doc["votos"]["ausencia"]) > 0):
            return True
        return False

    def _parse_html(self, html_content):
        tree = html.fromstring(html_content)
        
        title = tree.xpath("//span[contains(@id, 'ucLinkDocumento_lblDocumentoTitulo')]")
        if(len(title) == 0):
            logger.error("non existent")
            return

        pdfLink = tree.xpath("//a[contains(@id, 'ucLinkDocumento_hplDocumentoPDF')]")
        print("pdfLink",pdfLink)
        proposedBy = tree.xpath("//span[contains(@id, 'lblDeputadosGP')]")
        dataVotacao = tree.xpath("//span[contains(@id, 'lblData')]")
        resultadosVotacao = tree.xpath("//span[contains(@id, 'lblResultado')]")
        votosProposta = self._parse_voting_results(tree.xpath("//span[contains(@id, 'IniciativaFase_Detalhe_PLC1_ucVotacoes_rptVotacoes_ctl00_lblDetalhes')]"))
        comissao_elements = tree.xpath("//span[contains(@id, 'ucActividadeComissao_lblNome')]")
        
        authors = []
        authors_elements = tree.xpath("//a[contains(@id, 'hplAutor')]")
        for author in authors_elements:
            authors.append({"name":author.text, "bioURL":author.get("href")})

        comissoes = []
        comissoes_elements = tree.xpath("//span[contains(@id, 'ucActividadeComissao_lblNome')]")
        for comissao in comissoes_elements:
            comissoes.append(comissao.text)

        comissoes = list(set(comissoes))

        dataVotacao_s = dataVotacao[0].text
        dataVotacao_d = datetime.strptime(dataVotacao_s, '%Y-%m-%d')

        print("pdfLink",pdfLink)
        print("len(pdfLink)",len(pdfLink))

        if(len(pdfLink) == 0):
            return 

        doc = {
            "title":title[0].text,
            "pdfLink":pdfLink[0].get("href"),
            "dataVotacao":dataVotacao_d,
            "authors":authors,
            "comissoes":comissoes,
            "votos":votosProposta
        }
        
#         'votos': {'contra': [], 'afavor': [], 'abstencao': [], 'ausencia': []}
#         doc["has_voting_results"] = self._was_already_voted(doc)
        
        doc["anoVotacao"] = dataVotacao_d.year        
        
        if(len(proposedBy) > 0):
            doc["proposedBy"] = proposedBy[0].text

        elif(len(authors) > 0):
            doc["proposedBy"] = self._parse_party_from_author_name(authors[0]["name"])
            
        else:
            doc["proposedBy"] = "Governo"
            
        doc["resultadoFinal"] = None
        if(len(resultadosVotacao) > 0):
            doc["resultadoFinal"] = resultadosVotacao[-1].text_content()
        
        doc["has_voting_results"] = bool('resultadoFinal' in doc)
        
        return doc

import luigi
class HarvestConfig(luigi.Config):
    proposal_url_template = luigi.Parameter()
    webarchive_url_replay = luigi.Parameter()
    rscript_pdf_parser = luigi.Parameter()
    rscript_sampling_probabilities = luigi.Parameter()    

In [35]:


was_already_voted({'votos': {'contra': [], 'afavor': [], 'abstencao': [], 'ausencia': []}})

False

In [46]:
proposalsHarvester = PortugueseParlamentProposalsHarvester(HarvestConfig())
doc_proposal = proposalsHarvester.fetch_proposal("42995")
# doc_proposal = proposalsHarvester.fetch_proposal("42150")
doc_proposal

[__main__] INFO 2019-11-02 15:33:16 https://www.parlamento.pt/ActividadeParlamentar/Paginas/DetalheIniciativa.aspx?BID=42995


_handle_fetch_proposal_response
response.status_code 200
response.url https://www.parlamento.pt/ActividadeParlamentar/Paginas/DetalheIniciativa.aspx?BID=42995
pdfLink [<Element a at 0x11c29b590>]
pdfLink [<Element a at 0x11c29b590>]
len(pdfLink) 1


{'title': 'Recomenda ao Governo que reforce a formação dos profissionais de saúde na área da Geriatria, a nível pré-graduado; que diligencie junto da Ordem dos Médicos e da Ordem dos Enfermeiros, com vista à criação da especialidade de Geriatria nestas Ordens Profissionais; e que, uma vez criada a especialidade de Geriatria na Ordem dos Médicos e na Ordem dos Enfermeiros, reforce a formação dos profissionais de saúde nesta área, a nível pós-graduado.',
 'pdfLink': 'http://app.parlamento.pt/webutils/docs/doc.pdf?path=6148523063446f764c324679595842774f6a63334e7a637664326c756157357059326c6864476c3259584d7657456c4a535339305a58683062334d76634770794d5467774e53315953556c4a4c6d527659773d3d&fich=pjr1805-XIII.doc&Inline=true',
 'dataVotacao': datetime.datetime(2018, 9, 12, 0, 0),
 'authors': [{'name': 'Isabel Galriça Neto (CDS-PP)',
   'bioURL': '/DeputadoGP/Paginas/Biografia.aspx?BID=4096'},
  {'name': 'Teresa Caeiro (CDS-PP)',
   'bioURL': '/DeputadoGP/Paginas/Biografia.aspx?BID=1923'},
  {'na

In [12]:
for row in mongo_conn_database["proposals"].find().limit(10):
    print(row)

{'_id': ObjectId('5d17ac8e877f0c52f97e7968'), 'BID': 38660, 'anoVotacao': 2014, 'authors': [{'name': 'Paula Santos (PCP)', 'bioURL': '/noFrame/replay/20180101122541/http://www.parlamento.pt/DeputadoGP/Paginas/Biografia.aspx?BID=4197'}, {'name': 'Carla Cruz (PCP)', 'bioURL': '/noFrame/replay/20180101122541/http://www.parlamento.pt/DeputadoGP/Paginas/Biografia.aspx?BID=4297'}, {'name': 'João Oliveira (PCP)', 'bioURL': '/noFrame/replay/20180101122541/http://www.parlamento.pt/DeputadoGP/Paginas/Biografia.aspx?BID=2234'}, {'name': 'David Costa (PCP)', 'bioURL': '/noFrame/replay/20180101122541/http://www.parlamento.pt/DeputadoGP/Paginas/Biografia.aspx?BID=4447'}, {'name': 'Bruno Dias (PCP)', 'bioURL': '/noFrame/replay/20180101122541/http://www.parlamento.pt/DeputadoGP/Paginas/Biografia.aspx?BID=1715'}, {'name': 'Miguel Tiago (PCP)', 'bioURL': '/noFrame/replay/20180101122541/http://www.parlamento.pt/DeputadoGP/Paginas/Biografia.aspx?BID=2184'}, {'name': 'Paulo Sá (PCP)', 'bioURL': '/noFrame/r